<!--NOTEBOOK_HEADER-->
*This notebook contains material from [CBE60499](https://ndcbe.github.io/CBE60499);
content is available [on Github](git@github.com:ndcbe/CBE60499.git).*


<!--NAVIGATION-->
< [1.1 60 Minutes to Pyomo: An Energy Storage Model Predictive Control Example](https://ndcbe.github.io/CBE60499/01.01-Pyomo-Nuts-and-Bolts.html) | [Contents](toc.html) | [2.0 Optimization Modeling with Applications](https://ndcbe.github.io/CBE60499/02.00-Optimization-Modeling.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/01.02-Pyomo-Mini-Project.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/01.02-Pyomo-Mini-Project.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

In [ ]:
# IMPORT DATA FILES USED BY THIS NOTEBOOK
import os,  requests

file_links = [("data/Prices_DAM_ALTA2G_7_B1.csv", "https://ndcbe.github.io/CBE60499/data/Prices_DAM_ALTA2G_7_B1.csv")]

# This cell has been added by nbpages. Run this cell to download data files required for this notebook.

for filepath, fileurl in file_links:
    stem, filename = os.path.split(filepath)
    if stem:
        if not os.path.exists(stem):
            os.mkdir(stem)
    if not os.path.isfile(filepath):
        with open(filepath, 'wb') as f:
            response = requests.get(fileurl)
            f.write(response.content)


# 1.2 Pyomo Mini-Project: Receding Horizon Stochastic Control

<div style="background-color: rgba(255,0,0,0.05) ; padding: 10px; border: 1px solid darkred;"> 
<b>Important</b>: This assignment is a DRAFT. While the main ideas will not change, the notebook is still under development.
</div>

In [ ]:
# This code cell installs packages on Colab

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_ipopt()
    helper.download_data(['Prices_DAM_ALTA2G_7_B1.csv'])

In [1]:
import pandas as pd
import pyomo.environ as pyo
import numpy as np
import matplotlib.pyplot as plt

In the previous notebook, we considered model predictive control (MPC) of a battery energy storage system interacting with the electricity market. In essence, we posed an optimization problem:

*Given a forecasting for energy prices, what charging and discharging strategy maximizes net market revenue?*

In this homework mini-project, you will consider two modifications to make the example more realistic:
1. Add a constraint to prevent simultanous charging and discharging.
1. Receding horizon control
2. Uncertainty

The goal is to help you develop Python and Pyomo programming skills.

## 1.2.1 Model Refinement and Code Reorganization

### 1.2.1.1 Revised Model

Propose a new mathematical model that includes a constraint to prevent charging and discharging. Hint: using either integer variables or a disjunction will ensure the objective and constraints remaining linear.

Below is the model description copied from our in-class example. **Please edit** such that this notebook describes your complete new model.

**Sets**

Timesteps: $\mathcal{T} = \{0, 1, ..., N\}$
    
Timesteps without the initial time: $\mathcal{T}' = \{1, ..., N\}$

**Variables**
* $E_t$, energy stored in battery at time $t$, units: MWh
* $d_t$, battery discharge power (sold to market) during time interval [t-1, t), units: MW
* $c_t$, battery charge power (purchased from the market) during time interval [t-1, t), units: MW

**Parameters**
* $\pi_t$: Energy price during time interval [t-1, t), units: \$/MW
* $\eta$: Round trip efficiency, units: dimensionless
* $c_{max}$ Maximum charge power, units: MW
* $d_{max}$ Maximum discharge power, units: MW
* $E_{max}$ Maximum storage energy, units: MWh
* $E_{0}$ Energy in storage at time $t=0$, units: MWh

**Optimization Problem**
$$
\begin{align*}
    \max_{\mathbf{E},\mathbf{d},\mathbf{c}} \quad & \psi := \sum_{t \in \mathcal{T}'} \pi_{t} (d_{t} - c_{t}) \\
\mathrm{s.t.} \quad & E_{t} = E_{t-1} + c_{t} \sqrt{\eta} - \frac{d_{t}}{\sqrt{\eta}}, ~~ \forall ~ t \in \mathcal{T}' \\
    & E_{0} = E_{N} \\
    & 0 \leq c_{t} \leq c_{max}, ~~\forall ~ t \in \mathcal{T}' \\
    & 0 \leq d_{t} \leq d_{max}, ~~\forall ~ t \in \mathcal{T}' \\
    & 0 \leq E_{t} \leq E_{max},  ~~\forall ~ t \in \mathcal{T}'
\end{align*}
$$

### 1.2.1.2 Degree of Freedom Analysis

Perform degree of freedom analysis. Identify the (1) total number of variables, (2) total number of equality constraints, and (3) degrees of freedom. Tip: perform this on paper then typeset below.

Number of variables:
    
Number of (equality) constraints:
    
Degrees of freedom:

### 1.2.1.3 Python Code

Programmers organize their code into functions, often following the rule "1 task/action = 1 function". Here are a few of the reasons you should always do this:
1. Functions make your code easier to use
2. More specifically, functions force you to abstract/generalize your code
3. Writing doc strings (required for this course!) forces you to document your code and think about the structure.
4. Functions allow your code to best tested piece by piece

To show you how to do this, we have provided skeletons for several functions in this assignment. Based on the doc string, complete the functions.

In [ ]:
def build_model(price_data, horizon_length, include_pbc=True):
    # return m
    pass

def extract_solution(m):
    # return c_first_timestep, d_first_timestep, E_first_timestep
    pass

def plot_solution(t, c, d, E):
    #
    pass

### 1.2.1.4 Optimal Control Over 3 Day Horizon

Now let's use our functions for some analysis. Calculate the optimal battery operation strategy over 3 days starting on January 4, 2015.

#### 1.2.1.4.1 Original Model

#### 1.2.1.4.2 With Constraint Enforcing No Simultaneous Charging or Discharging

#### 1.2.1.4.3 Comparison

Write Python code below to compare the solutions. Print out any timesteps where the charge or discharge profile is different by more than 10^{-4} MW.

## 1.2.2 Receding Horizon Control

We now have functions to build, solve, and analyze the battery problem for a single problem. Now we can consider receeding horizon control.

*Please ask questions during class. You do not need to turn anything in for this section.*

Nomenclature: Let $u^*_{i}$ represent the optimal control action from time $t=i$ to $t=i+1$. For the battery system, $u_{i} = [c_i, d_i]^{T}$ (charging and discharging).

Algorithm:
1. Consider time $t=m$. 
2. Compute the optimal control actions $u^*=[u^*_m, u^*_{m+1}, ..., u^*_{m+N-1}]^T$ from time $t=m$ until $t=N+m$ where $N$ is the length of the planning horizon. In our battery example, we considered $N = 24$.
3. Simulate the system for one timestep using control action $u^*_{m}$.
4. (Optional). Update the system model or forecasts using new observations.
5. Step $t=m+1$ and goto Step 2.

### 1.2.2.1 Write Pseudocode

Pseudocode is an important planning tool for complex computer programs. The idea is to write out the code on paper or a whiteboard using generic programming syntax. As an example, consider calculating the 3rd through 10th elements of the Fibonacci sequence:

```
# Governing equation where f[i] is the ith element of the sequence
# f[i] = f[i-1] + f[i-2]

# Algorithm:

# Previous previous number in the sequence, f[i-2]
n_2prev = 1

# Previous number in the sequence, f[i-1]
n_1prev = 1

# Set counter
j = 3

while j <= 10 do:
    # Apply Fib. formula
    n = n1_prev + n2_prev
    
    # Print to screen
    print(n)
    
    # Shift history.
    # Previous number becomes previous previous number
    n2prev = n1prev
    
    # Current number becomes previous number
    n1prev = n
    
    # Increment counter
    j = j + 1
```

Here we sketched out the main algorithm (with comments!) in generic syntax. We did not worry about the correct way to write a while loop in Python. Once we figure out the algorithm we can worry about those details.

Write pseudocode for the receding horizon control example on paper or a whiteboard. Your algorithm should include the following main steps:
1. Load price data from a text file
2. Loop over timesteps. Keep track of implemented control actions (charge and discharge), battery SOC, and net revenue.
3. Make a plot of battery SOC and control actions that were actually implemented.

Scan your pseudocode as a PDF and turn in via Gradescope.

### 1.2.2.2 Python Code

In [ ]:
def update_model(m, new_price_data, new_EO):
    pass

def receding_horizon_simulation(start_hour, simulation_length):
    # return c_implemented, d_implemented, E_implemented
    pass

### 1.2.2.3 Simulate Performance Over 1 Week

Perform receding horizon control for the first 5 days in January 2015. Make a plot to show the implemented control action.

Tips:
* The implemented control action is a concationation of the first timestep for each optimization problem.
* How can you resuse the plot function above?

### 1.2.2.4 Impact of Horizon Length

Repeat the sensitivity analysis for horizon lengths of N=2 through N=48. For each simulation, compute the following:
1. Revenue from first 30 days
2. Total energy purchased from the market during the first 30 days
3. Total energy sold to the market during the first 30 days
4. Total computation time

Make four well labeled plots to show these trends.

**Discussion** How long of a simulation horizon do you recommend? Discuss the trade-off between revenue and computation time.

### 1.2.2.5 Impact of Periodic Boundary Condition

We will now explore the impact of the periodic boundary constraint.

First, simulate the performance over 1 week without enforcing the periodic boundary constraint. Plot the implemented control action and calculate the total revenue.

**Discussion** Compare the control actions and revenue with and without the periodic boundary constraint. How are they different (if at all). Offer an explanation for any differences.

Next, repeat the horizon length sensitivity analysis without the peridoic boundary constraint.

**Discussion** Compare these result to the analysis with periodic boundary constraints. Based on these results, make a recommendation to a battery operator.

## 1.2.3 Forecast Uncertainty (required for CBE 60499, optional for CBE 40499)
Generate white noise with mean zero and standard deviation 5 $/MWh using `np.random.normal()` and add this to the price forecast. Calculate how much less revenue you make with price uncertainty compared to the perfect information case (Part B). *Hint:* Each time you run the simulation, you will get a different answer.

<!--NAVIGATION-->
< [1.1 60 Minutes to Pyomo: An Energy Storage Model Predictive Control Example](https://ndcbe.github.io/CBE60499/01.01-Pyomo-Nuts-and-Bolts.html) | [Contents](toc.html) | [2.0 Optimization Modeling with Applications](https://ndcbe.github.io/CBE60499/02.00-Optimization-Modeling.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/01.02-Pyomo-Mini-Project.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/01.02-Pyomo-Mini-Project.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>